# 使用 JSON 指令控制下位机
本产品使用大小脑架构开发，上位机通过串口（树莓派通过 GPIO 串口）将 JSON 格式的命令发送给下位机。
注意：本章课程作为后面介绍下位机JSON指令集的前置课程，内容于前面的 Python 底盘运动控制章节的内容相似，如果你已知了解过那个章节，可以简单了解下 JSON 数据格式的优点后直接学习 JSON 指令集。
## JSON 数据格式的优点
JSON（JavaScript Object Notation）是一种轻量级的数据交换格式，它已经成为互联网上数据传输的标准之一。以下是JSON的一些优势：
- 可读性强：JSON采用了一种易于人类理解和编写的文本格式，使用键值对的形式组织数据，这使得数据在传输和存储时更加易于阅读和理解。

- 轻量级：相比于XML等其他数据格式，JSON的语法更加简洁紧凑，因此它更加轻量级，能够减少数据传输的大小和网络带宽的占用，提高传输效率。

- 易于解析：JSON数据结构简单明了，易于解析和序列化，几乎所有的编程语言都提供了JSON的解析和生成库，使得开发人员可以方便地处理JSON数据。

- 与各种语言兼容性好：JSON几乎在所有编程语言中都有支持，因此可以很方便地在不同的平台和系统中进行数据交换和通信。

- 支持多种数据类型：JSON支持多种数据类型，包括字符串、数字、布尔值、数组和对象等，这使得它可以灵活地表示各种类型的数据结构。

- 与Web技术无缝结合：JSON最初是由JavaScript发展而来，因此与Web技术的集成非常紧密，它与JavaScript语言的兼容性非常好，可以很方便地在Web应用中使用。方便地在Web应用中使用。

## 简单的 JSON 指令控制下位机例程

在下面的例程中，我们使用 is_raspberry_pi5() 函数来判断当前的树莓派型号，因为树莓派4B和树莓派5的 GPIO 串口的设备名称是不同的，你需要使用正确的 GPIO 设备名称，且使用与下位机相同的波特率（默认为115200）。

运行以下代码块之前你需要先将产品架高起量，保持驱动轮全部离地，调用以下代码块后机器人会开始走动，小心不要让机器人从桌面上掉落。

In [ ]:
from base_ctrl import BaseController
import time

# 用于检测树莓派的函数
def is_raspberry_pi5():
    with open('/proc/cpuinfo', 'r') as file:
        for line in file:
            if 'Model' in line:
                if 'Raspberry Pi 5' in line:
                    return True
                else:
                    return False

# 根据树莓派的型号来确定 GPIO 串口设备名称
if is_raspberry_pi5():
    base = BaseController('/dev/ttyAMA0', 115200)
else:
    base = BaseController('/dev/serial0', 115200)

# 轮子以0.2m/s的速度转动2秒钟后停止
base.send_command({"T":1,"L":0.2,"R":0.2})
time.sleep(2)
base.send_command({"T":1,"L":0,"R":0})

通过调用上面的代码块，树莓派会首先发送 {"T":1,"L":0.2,"R":0.2} 这条指令（后面章节我们会再具体介绍指令的构成），车轮开始转动，间隔两秒钟后树莓派会发送 {"T":1,"L":0,"R":0} 这条指令，车轮会停止转动，这里需要注意的一点是，即使不发送后面的停止车轮转动的指令，如果你没有发送新的指令，车轮依然会停止转动，这是因为下位机内含有心跳函数，心跳函数的做用是在上位机长时间没有新的指令发送给下位机时，下位机自动停止目前的移动指令，改函数的目的是为了避免上位机由于某些原因死机而导致下位机继续运动。

如果你希望机器人一直持续不断地运动下去，上位机需要每隔2秒-4秒循环发送运动控制的指令。